# **<font color=#0abab5>Process and Concurrency</font>**

repo : [https://github.com/Harrison-Chen/python-study-group-ch15](https://github.com/Harrison-Chen/python-study-group-ch15)

### Thread and Processes

複習一下os必考題

OpenGPT:


```
程序通常都是由一個或多個進程組成的。進程是計算機內存中的一個執行實例，具有它自己的代碼和數據。它們可以平行執行以提高程序的效率。

執行緒（thread）是進程中的一個執行流。它是一個獨立的執行路徑，可以在進程的內存空間中執行。執行緒可以平行執行以提高程序的效率。

進程和執行緒之間的主要區別在於它們的執行環境。進程擁有完整的執行環境，包括它自己的內存空間和資源。執行緒則共享進程的內存空間和資源。因此，執行緒更加輕量級，但也受到進程的限制。

此外，進程之間是相互隔離的，它們之間不能直接交互。相反，它們必須通過特定的技術，例如共享記憶體或消息傳遞，才能進行通信。執行緒則可以直接訪問進程中的共享資源，因此它們之間的交互更加方便。

總之，進程和執行緒都是用於提高程序效率的工具。選擇哪種方法取決於您的程序需求和目標。
```

### Create a process with subprocess

程式碼預設是用一個process去進行，但可以利用os庫的api從process上創造其他的process叫subprocess

下面這段利用subprocess程式庫呼叫unix的date，抓取stdout放到變數

底下放了很多種用法，可以參考使用

In [6]:
import subprocess
ret = subprocess.getoutput('date')
ret

'Thu Dec  8 21:42:36 CST 2022'

In [5]:
ret = subprocess.getoutput('date -u')
ret

'Thu Dec  8 13:42:32 UTC 2022'

In [3]:
# or with pipe
ret = subprocess.getoutput('date -u | wc')
ret

'       1       6      29'

In [7]:
# check_output method
ret = subprocess.check_output(['date', '-u'])
ret

b'Thu Dec  8 13:42:57 UTC 2022\n'

In [12]:
# exit code
ret = subprocess.getstatusoutput('date')
ret

(0, '2022年12月 2日 週五 20時37分16秒 CST')

In [14]:
# dont care output but exit code
ret = subprocess.call('date')
ret

2022年12月 2日 週五 20時38分41秒 CST


0

In [18]:
# in shell script format
ret = subprocess.call('date -u', shell = True)

# list of arg way
ret = subprocess.call(['date', '-u'])

2022年12月 2日 週五 12時40分50秒 UTC
2022年12月 2日 週五 12時40分50秒 UTC


### Create process with multiprocessing

用另外一個multiprocess的庫去跑

但是jupyter(ipython)上面跑起來會有錯誤

解法是要從另外一個檔案連結過來跑，但只是短demo，所以乾脆直接在另一個檔案上面跑

In [13]:
# 不能在juypter跑，另外跑
from multiprocessing import Process
import os

def whoami(what):
    print("Process %s says: %s" % (os.getpid(), what))

if __name__ == "__main__": 
    whoami("I'm the main program") 
    for n in range(4):
        p = Process(target=whoami,
            args=("I'm function %s" % n,))
        p.start()

multipreocessing module非常實用，可以用在所有外部程式的呼叫以及把資料跟python腳本互動

在concurrency段落有更多應用

#### Kill Process

multiprocess有提供terminate接口，可以呼叫os call進行kill process

In [ ]:
# kill process

# 另外執行
import multiprocessing
import time
import os
def whoami(name):
    print("I'm %s, in process %s" % (name, os.getpid()))

def loopy(name): 
    whoami(name)
    start = 1
    stop = 1000000
    for num in range(start, stop):
        print("\tNumber %s of %s. Honk!" % (num, stop))
        time.sleep(1)

if __name__ == "__main__": 
    whoami("main")
    p = multiprocessing.Process(target=loopy, args=("loopy",))
    p.start()
    time.sleep(5)
    p.terminate()

### Command Automation

用python來管理、執行類似shell script用途的操作

#### Invoke
把function呼叫轉成command line arguments

In [ ]:
import os
os.system('pip install invoke')

In [43]:
# run my cli
from invoke import task

@task
def mytime(ctx): 
    import time
    now = time.time()
    time_str = time.asctime(time.localtime(now)) 
    print("Local time is", timestr)

### Concurrency

平行處理（concurrency）指的是可以同時進行多個任務的能力。這本書舉例了兩個最常見的需要平行處理的情境：

- I/O bound：等待網路或硬碟的輸入/輸出操作。

- CPU/GPU bound：CPU/GPU在計算一個大的，導致他們一直在忙碌，圖學跟科學計算都是常見情境。

此外，這段描述還提到了兩個重要的術語：同步（synchronous）和非同步（asynchronous）。

同步指的是連續進行的任務，而非同步指的是各自獨立運行的任務。

總而言之，這段描述指出平行處理是一種解決無法等待的情境的方法。

與其在那裡忙碌等待(busy-waiting)，不如直接平行處理，做其他服務應該做的事情。

### Queues

multiprocessing module有著queue function存在，

可以將process的呼叫queue起來

範例如下：

In [ ]:
# 也要另外跑
import multiprocessing as mp

def washer(dishes, output): 
    for dish in dishes:
        print('Washing', dish, 'dish') 
        output.put(dish)

def dryer(input): 
    while True:
        dish = input.get() 
        print('Drying', dish, 'dish') 
        input.task_done()

if __name__ == '__main__':
    dish_queue = mp.JoinableQueue()
    dryer_proc = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc.daemon = True
    dryer_proc.start()

    dishes = ['salad', 'bread', 'entree', 'dessert']
    washer(dishes, dish_queue)
    dish_queue.join()

### Threads

Thread是在同個process中執行，一對多的資源單元
跟上面multiprocessing module對應，有著threading的module

以下用thread進行類似上面process的任務

In [53]:
import threading 

def do_this(what):
    whoami(what) 

def whoami(what):
    print("Thread %s says: %s" % (threading.current_thread(), what))

if __name__ == "__main__": 
    whoami("I'm the main program") 
    for n in range(4):
        p = threading.Thread(target=do_this,
                              args=("I'm function %s" % n,))
        p.start()   

Thread <_MainThread(MainThread, started 4313335168)> says: I'm the main program
Thread <Thread(Thread-22, started 6364196864)> says: I'm function 0
Thread <Thread(Thread-23, started 6364196864)> says: I'm function 1
Thread <Thread(Thread-24, started 6364196864)> says: I'm function 2
Thread <Thread(Thread-25, started 6364196864)> says: I'm function 3


In [ ]:
import threading, queue 
import time

def washer(dishes, dish_queue): 
    for dish in dishes:
        print ("Washing", dish) 
        time.sleep(5)
        dish_queue.put(dish)
def dryer(dish_queue): 
    while True:
        dish = dish_queue.get() 
        print ("Drying", dish) 
        time.sleep(10) 
        dish_queue.task_done()

dish_queue = queue.Queue() 
for n in range(2):
    dryer_thread = threading.Thread(target=dryer, args=(dish_queue,))
    dryer_thread.start()

dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)

dish_queue.join()


Threads可能會產生危險的結果，多個thread可能會導致code難以debug。因此，您需要確保在thread中使用的所有API都是thread safe的。

在上面的範例中，因為沒有考慮到thread safe，所以執行順序都亂成一團。

在使用multi-process的時候，可以確保每個process都是獨立運行的，並且在獨立結束時回報結果。

整個process是一整組的，在完成運行後才會回報結果。

#### Thread safe

當thread沒有使用到global data時，它是安全且好用的。特別是在完成IO操作時，thread可以比process快很多。

但是，在許多情況下，thread需要控制一些global data。在這種情況下，thread safe就非常重要，以確保避免preemption。

安全地共享資料的一種方法是使用software lock。在lock段落中變更參數時，會禁止其他thread操作該段落。這樣，就可以確保資料在被變更時不會受到其他thread的干擾。

#### CPU-bound

書上提到python的thread沒辦法解決cpu-bound的問題，因為python interperter的Global Interpreter Lock設計

所以跟傳統thread加速不一樣，python的multi-thread可能比single-thread或multi-process更慢

所以書上建議：

- thread 只用來處理 I/O bound problem
- CPU bound用processes, networking or events去解決

### concurrent.futures

一個thread跟multiple的module用法，可以更方便的schedule async pooled workers

照上面所述去解決各自的問題

In [ ]:
# cf1.py
from concurrent import futures 
import math
import time
import sys
def calc(val): 
    time.sleep(1)
    result = math.sqrt(float(val)) 
    return result

def use_threads(num, values): 
    t1 = time.time()
    with futures.ThreadPoolExecutor(num) as tex: 
        results = tex.map(calc, values)
    t2 = time.time() 
    return t2 - t1

def use_processes(num, values): 
    t1 = time.time()
    with futures.ProcessPoolExecutor(num) as pex: 
        results = pex.map(calc, values)
    t2 = time.time() 
    return t2 - t1

def main(workers, values):
    print(f"Using {workers} workers for {len(values)} values") 
    t_sec = use_threads(workers, values)
    print(f"Threads took {t_sec:.4f} seconds")
    p_sec = use_processes(workers, values)
    print(f"Processes took {p_sec:.4f} seconds")
    
if __name__ == '__main__':
    workers = 1
    values = list(range(1, 6)) # 1 .. 5 main(workers, values)
    main(workers, values)

In [ ]:
# cf2.py
from concurrent import futures 
import math
import sys

def calc(val):
    result = math.sqrt(float(val)) 
    return val, result

def use_threads(num, values):
    with futures.ThreadPoolExecutor(num) as tex:
        tasks = [tex.submit(calc, value) for value in values] 
        for f in futures.as_completed(tasks):
            yield f.result()

def use_processes(num, values):
    with futures.ProcessPoolExecutor(num) as pex:
        tasks = [pex.submit(calc, value) for value in values] 
        for f in futures.as_completed(tasks):
            yield f.result()

def main(workers, values):
    print(f"Using {workers} workers for {len(values)} values") 
    print("Using threads:")
    for val, result in use_threads(workers, values): 
        print(f'{val} {result:.4f}')
    print("Using processes:")
    for val, result in use_processes(workers, values):
        print(f'{val} {result:.4f}')

if __name__ == '__main__': 
    workers = 3
    if len(sys.argv) > 1:
        workers = int(sys.argv[1])
    values = list(range(1, 6)) # 1 .. 5 
    main(workers, values)

### Green Threads and gevent

openGPT解釋：
```
Python的Green Threads是一種用戶級的線程實現，它允許用戶在單個進程中創建多個線程。它與操作系統提供的線程不同，操作系統線程是由操作系統調度的，而Green Threads是由Python解釋器調度的。

Green Threads在Python 2.6中引入，但在Python 3.2中被標記為廢弃。在Python 3.2和更高版本中，Python使用操作系統提供的線程。

gevent是一個用於網絡應用程序的庫，它使用Green Threads來處理並發。它通過提供一組工具，使得開發人員可以在不使用多線程的情況下實現並發。它可以幫助開發人員更輕鬆地處理高並發的網絡應用程序，因為它可以避免多線程程序設計中的一些常見問題，如競爭條件和死鎖。
```

平行化是一種在多核處理器上運行程序的方法，用於加速程序運行。 Apache是一個支持多線程的Web服務器，它可以使用多個線程來同時處理多個請求。

另一方面，事件驅動程序通過使用中心事件循環來分配任務和進行工作。 NGINX是一個基於事件的Web服務器，它可以通過使用單個線程來高效地處理大量請求。由於它使用的是單線程，

因此通常比多線程的Web服務器（如Apache）更快。

In [ ]:
import os

os.system('pip install gevent')

In [19]:
import gevent
from gevent import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
'www.google.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts] 
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)

66.6.44.4
144.217.51.126
172.217.163.36


gevent提供了一種使用Green Threads的方法，該方法可以通過生成greenlet（green thread/microthread）來實現平行。

這些greenlet各自運行其事件，然後在完成後收集結果。

Green Threads和OS提供的線程有所不同。 Green Threads不會阻塞，因此在等待時，gevent會切換到其他greenlets。 Greenlets之間主要通過yield來進行交互，而不是通過搶占資源。

Green Threads的實現方式與fiber類似，因為它們都是用戶級的線程實現，並且由解釋器調度。

#### gevent.monkey

gevent提供了一個 monkey 模塊，其中包含一種方法（ patch_socket ），可以替換掉程式中的 socket 呼叫，以便使用 gevent 版本。

使用方法如下：

```
from gevent import monkey
monkey.patch_socket()
```

呼叫上面這行，會讓程式中所有socket呼叫變成gevent版本

請注意，這種方法只會替換 Python 寫的函數集。對於用 C 語言寫的函數，它不會進行替換。

In [65]:
import gevent
from gevent import monkey; monkey.patch_all()
import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
'www.google.com']

jobs = [gevent.spawn(socket.gethostbyname, host) for host in hosts] 

gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)

66.6.44.4
144.217.51.126
172.217.163.36


#### gevent的問題

和所有event-based的程式碼一樣，每一個指派給gevent的任務都應該相對快速。

儘管相對於多線程，gevent具有非阻塞特性，但在其上進行大量運算仍然可能很慢。

儘管 monkey-patching 的大量替換方式可能會感覺有點危險，但許多大型網站（如 Pinterest）都使用 gevent 來加速他們的網站。最好多查看 gevent 的官方指引，以避免使用錯誤。





### Twisted

twisted 是一個非同步、事件驅動的網路框架，可以聆聽一些事件的觸發來執行function。

是一個callback design

In [ ]:
import os

os.system('pip install twisted')

In [ ]:
# knock-server

from twisted.internet import protocol, reactor

class Knock(protocol.Protocol): 
    def dataReceived(self, data):
        print('Client:', data)
        if data.startswith("Knock knock"):
            response = "Who's there?" 
        else:
            response = data + " who?" 
            print('Server:', response) 
            self.transport.write(response)
class KnockFactory(protocol.Factory): 
    def buildProtocol(self, addr):
        return Knock()

reactor.listenTCP(8000, KnockFactory())
reactor.run()

In [ ]:
# knock-client

from twisted.internet import reactor, protocol

class KnockClient(protocol.Protocol): 
    def connectionMade(self):
        self.transport.write("Knock knock")
    def dataReceived(self, data):
        if data.startswith("Who's there?"):
            response = "Disappearing client"
            self.transport.write(response) 
        else:
            self.transport.loseConnection()
            reactor.stop()
class KnockFactory(protocol.ClientFactory): 
    protocol = KnockClient

def main():
    f = KnockFactory() 
    reactor.connectTCP("localhost", 8000, f) 
    reactor.run()

if __name__ == '__main__': 
    main()

### asyncio

python在3.4加入官方library中，大家的最愛async await

比起callback更好可讀性，以及更直覺的呼叫await操作

In [84]:
import asyncio

async def say(phrase, seconds):
    print(phrase)
    await asyncio.sleep(seconds)

async def wicked():
    task_1 = asyncio.create_task(say("Surrender,", 2))
    task_2 = asyncio.create_task(say("Dorothy!", 0))
    await task_1
    await task_2

await wicked()

await say("hello", 2)
await say("world", 0)

Surrender,
Dorothy!
hello
world


#### Redis

用python與redis資料庫互動，更有效率的用in memory db做存取

這本書這邊提到是想講解跨機器的平行處理的方法論

不過redis的操作在下一章才有，這邊先跳過

#### Beyond Queues

在high-level的系統設計上，平行處理的系統可能會遇到一些問題。
通常的解決方案包括：

- 火放並忘記（Fire and forget）：把執行的事情丟下去就不管了。
- 請求回復（Request-reply）：對於每個請求都要等到完成的確認。
- 壓力反應或調速（Back pressure or throttling）：根據負載調節工作程序的使用。


有一些套件可以負責管理這些最高層的佇列。例如：

celery：利用上面提到的 gevent、process 等來執行分散式的任務。
rq：python 的工作佇列庫，基於 redis。

## Discussion
[https://docs.google.com/spreadsheets/d/1va_JEoP7RNDqNpUGJCBC6cstIWQCCrFpz0WmzmRXY-8/edit#gid=0
](https://docs.google.com/spreadsheets/d/1va_JEoP7RNDqNpUGJCBC6cstIWQCCrFpz0WmzmRXY-8/edit#gid=0
)
- 遇過什麼印象深刻的多執行緒程式碼範例

- 在其他語言中process/thread/green-thread(fiber)的應用跟python有什麼相同與不同之處